In [ ]:
# Uploading libraries
import ast
import pandas as pd
import seaborn as sns
import numpy as np
from datasets import load_dataset
import datetime
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from adjustText import adjust_text


# Uploading dataset
df_2019_q2 = pd.read_csv('Original_data_Q1_2019_Q1_2020/Divvy_Trips_2019_Q2.csv')



In [6]:
df_2019_q2.sample(10)

,01 - Rental Details Rental ID,01 - Rental Details Local Start Time,01 - Rental Details Local End Time,01 - Rental Details Bike ID,01 - Rental Details Duration In Seconds Uncapped,03 - Rental Start Station ID,03 - Rental Start Station Name,02 - Rental End Station ID,02 - Rental End Station Name,User Type,Member Gender,05 - Member Details Member Birthday Year
689747,22989942,2019-06-04 20:26:34,2019-06-04 20:34:04,2081,450.0,140,Dearborn Pkwy & Delaware Pl,359,Larrabee St & Division St,Subscriber,Male,1955.0
436409,22693179,2019-05-16 18:49:29,2019-05-16 18:56:33,2730,424.0,318,Southport Ave & Irving Park Rd,330,Lincoln Ave & Addison St,Subscriber,Male,1980.0
646433,22939484,2019-06-02 10:38:34,2019-06-02 13:06:48,1902,"8,894.0",176,Clark St & Elm St,307,Southport Ave & Clybourn Ave,Customer,NaN,NaN
442816,22700828,2019-05-17 11:50:34,2019-05-17 11:56:15,3777,341.0,210,Ashland Ave & Division St,628,Walsh Park,Subscriber,Male,1992.0
844843,23170212,2019-06-14 15:13:46,2019-06-14 15:33:26,5392,"1,180.0",103,Clinton St & Polk St (*),146,Loomis St & Jackson Blvd,Customer,Female,1988.0
239986,22459984,2019-04-26 15:08:46,2019-04-26 15:18:13,4597,567.0,40,LaSalle St & Adams St,110,Dearborn St & Erie St,Subscriber,Male,1989.0
372872,22617651,2019-05-11 11:21:24,2019-05-11 11:26:36,1859,312.0,143,Sedgwick St & Webster Ave,141,Clark St & Lincoln Ave,Subscriber,Female,1975.0
407047,22658580,2019-05-14 19:30:08,2019-05-14 19:44:54,4269,886.0,300,Broadway & Barry Ave,141,Clark St & Lincoln Ave,Subscriber,Male,1989.0
842512,23167699,2019-06-14 13:19:40,2019-06-14 13:25:59,5384,379.0,230,Lincoln Ave & Roscoe St,227,Southport Ave & Waveland Ave,Subscriber,Male,1966.0
541904,22817061,2019-05-25 14:01:22,2019-05-25 14:43:31,5042,"2,529.0",6,Dusable Harbor,90,Millennium Park,Customer,NaN,NaN


In [8]:
df_2019_q2['User Type'] = df_2019_q2['User Type'].replace('Customer', 'casual')
df_2019_q2['User Type'] = df_2019_q2['User Type'].replace('Subscriber', 'member')

In [9]:
# Dropped unused calumns

columns_to_drop = ['01 - Rental Details Bike ID', '01 - Rental Details Duration In Seconds Uncapped',	'03 - Rental Start Station ID',	'03 - Rental Start Station Name',	'02 - Rental End Station ID',	'02 - Rental End Station Name',	'Member Gender',	'05 - Member Details Member Birthday Year']

df_clean_2019_q2 = df_2019_q2.drop(columns=columns_to_drop)

df_clean_2019_q2.sample(5)

,01 - Rental Details Rental ID,01 - Rental Details Local Start Time,01 - Rental Details Local End Time,User Type
355392,22597284,2019-05-09 17:15:19,2019-05-09 17:21:38,member
584753,22867971,2019-05-29 08:16:04,2019-05-29 08:29:01,member
36587,22222306,2019-04-05 17:32:21,2019-04-05 17:37:34,member
806556,23125313,2019-06-11 19:56:01,2019-06-11 20:01:41,member
774173,23087967,2019-06-10 08:08:48,2019-06-10 08:15:11,member


In [10]:
# trip_id	start_time	end_time	usertype
# Renamed columns to make mergeable with other datasets in analysis project

column_mapping = {'01 - Rental Details Rental ID': 'trip_id',
                  '01 - Rental Details Local Start Time': 'start_time',
                  '01 - Rental Details Local End Time': 'end_time',
                  'User Type': 'usertype'
                  }
df_clean_2019_q2.rename(columns=column_mapping, inplace=True)

df_clean_2019_q2.sample(5)

,trip_id,start_time,end_time,usertype
388757,22637430,2019-05-13 16:57:20,2019-05-13 17:15:32,member
816396,23137114,2019-06-12 16:51:10,2019-06-12 17:06:24,member
1092207,23461002,2019-06-29 20:28:29,2019-06-29 21:02:32,member
693519,22994217,2019-06-05 08:23:45,2019-06-05 08:30:26,member
1081027,23448096,2019-06-29 13:22:22,2019-06-29 16:09:52,casual


In [11]:
# Formatted columns start_time & end_time from Series to date&time then created new column where calculated trip duration

df_clean_2019_q2['start_time'] = pd.to_datetime(df_clean_2019_q2['start_time'], format='%Y-%m-%d %H:%M:%S')
df_clean_2019_q2['end_time'] = pd.to_datetime(df_clean_2019_q2['end_time'], format='%Y-%m-%d %H:%M:%S')


df_clean_2019_q2['ride_length_seconds'] = (df_clean_2019_q2['end_time'] - df_clean_2019_q2['start_time']).dt.total_seconds()

df_clean_2019_q2['ride_length_seconds'] = df_clean_2019_q2['ride_length_seconds'].astype(int)


def seconds_to_hhmmss(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"


df_clean_2019_q2['ride_length'] = df_clean_2019_q2['ride_length_seconds'].apply(seconds_to_hhmmss)


df_clean_2019_q2.sample(10)

,trip_id,start_time,end_time,usertype,ride_length_seconds,ride_length
694278,22995070,2019-06-05 08:52:11,2019-06-05 09:07:14,member,903,00:15:03
81071,22273246,2019-04-09 18:58:44,2019-04-09 19:20:40,member,1316,00:21:56
1001552,23353973,2019-06-25 11:50:31,2019-06-25 12:23:43,member,1992,00:33:12
125909,22326396,2019-04-16 08:17:50,2019-04-16 08:22:57,member,307,00:05:07
510954,22781531,2019-05-23 11:45:15,2019-05-23 11:52:17,member,422,00:07:02
698492,23000170,2019-06-05 15:10:49,2019-06-05 15:18:18,member,449,00:07:29
1506,22180195,2019-04-01 08:29:00,2019-04-01 08:33:12,member,252,00:04:12
585341,22868704,2019-05-29 08:42:15,2019-05-29 08:59:04,casual,1009,00:16:49
667338,22963831,2019-06-03 15:08:10,2019-06-03 15:39:37,casual,1887,00:31:27
424337,22678732,2019-05-15 21:11:28,2019-05-15 21:33:52,member,1344,00:22:24


In [12]:
df_clean_2019_q2['day_of_week'] = df_clean_2019_q2['start_time'].dt.day_name()

df_clean_2019_q2.sample(10)


,trip_id,start_time,end_time,usertype,ride_length_seconds,ride_length,day_of_week
460283,22721375,2019-05-19 11:03:00,2019-05-19 11:30:58,member,1678,00:27:58,Sunday
722807,23028480,2019-06-06 19:13:54,2019-06-06 19:27:31,member,817,00:13:37,Thursday
606926,22894104,2019-05-30 17:12:40,2019-05-30 17:33:56,member,1276,00:21:16,Thursday
398819,22649314,2019-05-14 12:22:26,2019-05-14 12:26:16,member,230,00:03:50,Tuesday
696792,22998183,2019-06-05 12:49:10,2019-06-05 13:17:33,member,1703,00:28:23,Wednesday
1091461,23460169,2019-06-29 19:47:06,2019-06-29 19:53:57,member,411,00:06:51,Saturday
93363,22288323,2019-04-11 15:59:08,2019-04-11 16:19:16,casual,1208,00:20:08,Thursday
691003,22991435,2019-06-04 22:27:29,2019-06-04 22:33:14,member,345,00:05:45,Tuesday
374848,22620137,2019-05-11 15:42:27,2019-05-11 15:55:56,member,809,00:13:29,Saturday
658567,22953110,2019-06-02 19:15:28,2019-06-02 19:22:56,member,448,00:07:28,Sunday


In [13]:
df_clean_2019_q2.to_csv('Cleaned_2019_Q2.csv', index=False)